In [0]:

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
class Transformer:
    def __init__(self):
        pass
    def transform(self,inputDFs):
        pass
    
class AirpodsAfterIphone(Transformer):
    def transform(self,inputDFs):
       """ 
       customer who bought airpods after buying the iphone
       """
       transaction_input_df = inputDFs.get("transaction_input_df")
       print("transaction_input_df in transform")
       transaction_input_df.display()
       

       windowspec = Window.partitionBy('customer_id').orderBy('transaction_date')
       transformdf = transaction_input_df.withColumn("next_product_name",lead("product_name").over(windowspec))

       print("airpods after buying iphone")

       transformdf.orderBy('customer_id','transaction_date','product_name').display()
       print("filter df !!!!") 

       filterdf = transformdf.filter((col('product_name') == 'iPhone') & (col('next_product_name') == 'AirPods'))
       filterdf.orderBy('customer_id','transaction_date','product_name').display() 

       customer_input_df = inputDFs.get('customer_input_df')
       join_df = customer_input_df.join(broadcast(filterdf),"customer_id")

       print("join df !!!!!")
       join_df.display()
       return join_df.select('customer_id','customer_name','location')
   

class OnlyAirpodsAndIphone(Transformer):
    def transform(self,inputDFs):
        """
        customer who bought only iphone and airpods nothing else
        """
        transaction_input_df = inputDFs.get("transaction_input_df")
        print("transaction_input_df in transform")
        transaction_input_df.display()

        groupdf = transaction_input_df.groupBy("customer_id").agg(collect_set("product_name").alias("produts"))
        groupdf.display()

        filterdf = groupdf.filter((array_contains(col('produts'),"iPhone")) & 
                                  (array_contains(col('produts'),"AirPods"))&
                                  (size(col("produts")) == 2))
        filterdf.display() 

        customer_input_df = inputDFs.get('customer_input_df')
        join_df = customer_input_df.join(broadcast(filterdf),"customer_id")

        print("join df !!!!!")
        join_df.display()
        return join_df.select('customer_id','customer_name','location')
        